In [1]:
import pandas as pd
import seaborn as sns
import os

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
print(os.getcwd())

/Users/matheus/Projects/steam_analysis/steam-analysis/notebooks/analysis


In [4]:
os.chdir("../..")

In [5]:
from src.features.build_features import (
    CreateTargetColumns,
    CreateBinaryColumns,
    DateColumnsTransformer,
    DeleteNaNRows,
    CounterTransformer,
    YearThresholdTransformer,
    IndieTransformer,
)

In [6]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    steps=[
        ("drop_NaN_columns", DeleteNaNRows()),
        ("create_binary_columns", CreateBinaryColumns()),
        ("create_counter_columns", CounterTransformer()),
        ("create_target_columns", CreateTargetColumns(threshold=500)),
        ("create_date_columns", DateColumnsTransformer()),
        ("filter_dataset_year", YearThresholdTransformer()),
        ("filter_dataset_indie_only", IndieTransformer()),
    ]
)

In [7]:
steam_df = pd.read_csv("data/raw/games.csv")

In [8]:
steam_df.columns = [col.lower().replace(" ", "_") for col in steam_df.columns]

In [9]:
pipeline.fit_transform(steam_df)

,appid,name,release_date,estimated_owners,peak_ccu,required_age,price,dlc_count,about_the_game,supported_languages,full_audio_languages,reviews,header_image,website,support_url,support_email,windows,mac,linux,metacritic_score,metacritic_url,user_score,positive,negative,score_rank,achievements,recommendations,notes,average_playtime_forever,average_playtime_two_weeks,median_playtime_forever,median_playtime_two_weeks,developers,publishers,categories,genres,tags,screenshots,movies,has_publisher,total_reviews,perceived_quality,has_support_email,has_support_url,has_website,n_screens,n_movies,n_tags,n_languages,target_success,month,day,year
68042,1364760,COTTOn Rock'n'Roll -SUPERLATIVE NIGHT DREAMS-,2023-01-05,0 - 20000,12,0,22.49,0,The original beautiful girl side-scrolling sho...,"{ 'Traditional Chinese', 'Japanese', 'English...","['Japanese', 'Simplified Chinese']",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,https://cotton-rock-n-roll.success-corp.co.jp/,https://www.success-corp.co.jp/inquiry/consume...,NaN,True,False,False,0,NaN,0,10,0,NaN,23,0,NaN,0,0,0,0,Studio Saizensen,SUCCESS Corp.,"Single-player,Steam Achievements,Full controll...",Action,"Action,Arcade,Shooter,Shoot 'Em Up,Female Prot...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,True,10,10.000000,False,True,True,10,2,6,9,False,1,5,2023
68049,2185760,Under the Sky World,2023-01-04,0 - 20000,14,0,14.39,0,'Under the Sky World' is an orthodox casual RP...,"{ 'English', 'Traditional Chinese', 'Japanese...",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,bokibokiofficial@gmail.com,True,False,False,0,NaN,0,10,0,NaN,0,0,NaN,0,0,0,0,サークル☆フェアリーフラワー,BokiBoki Games,Single-player,"Adventure,Casual,Indie,RPG","JRPG,Sexual Content,Casual,Hentai,Female Prota...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,True,10,10.000000,True,False,False,9,2,17,4,False,1,4,2023
68066,2088160,天外武林 (Traveler of Wuxia),2023-01-05,20000 - 50000,5707,0,22.49,0,Traveler of Wuxia is a deck-buiding roguelite ...,"{ 'Traditional Chinese', 'Simplified Chinese'}",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,https://www.facebook.com/heluogame/,NaN,True,False,False,0,NaN,0,702,165,NaN,37,878,NaN,427,427,427,427,香港商河洛互動娛樂股份有限公司,香港商河洛互動娛樂股份有限公司,"Single-player,Steam Achievements,Steam Cloud","RPG,Strategy","Rogue-like,Deckbuilding,Card Game,RPG,Card Bat...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,False,867,4.228916,False,True,False,11,1,19,2,True,1,5,2023
68069,2209560,NoEL - Nothing on Elysion Line,2023-01-05,0 - 20000,6,0,17.49,0,"게임 기본 정보 =============== ◎ 전투를 통해 덱을 수정하고, 장비를...",{'Korean'},['Korean'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,xenogames00@gmail.com,True,False,False,0,NaN,0,7,3,NaN,0,0,NaN,0,0,0,0,Xenogames,Xenogames,Single-player,"Adventure,Casual,Indie,RPG,Strategy","Card Game,Rogue-like,Anime,Visual Novel,Roguel...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,False,10,1.750000,True,False,False,8,1,20,1,False,1,5,2023
68074,1816070,ILLUMINATI,2023-01-06,0 - 20000,7,0,21.24,0,Battle for world domination in this darkly sat...,{'English'},['English'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,https://illuminati.game,https://illuminati.game,support@radiofree.net,True,True,False,0,NaN,0,11,5,NaN,0,0,"Some depictions of bloody violence, mortal per...",0,0,0,0,"Derek Pearcy,Matt Seifert",RadioFree.Net,"Single-player,Multi-player,PvP,Online PvP,Cros...","Casual,Indie,Strategy","Strategy,Card Battler,PvP,Casual,Card Game,2D,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,True,16,1.833333,True,True,True,7,1,17,1,False,1,6,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [8]:
teste = DeleteNaNRows()

In [9]:
steam_df = teste.fit_transform(steam_df)

In [10]:
teste = CounterTransformer()

In [11]:
steam_df = teste.fit_transform(steam_df)

In [12]:
steam_df

,appid,name,release_date,estimated_owners,peak_ccu,required_age,price,dlc_count,about_the_game,supported_languages,full_audio_languages,reviews,header_image,website,support_url,support_email,windows,mac,linux,metacritic_score,metacritic_url,user_score,positive,negative,score_rank,achievements,recommendations,notes,average_playtime_forever,average_playtime_two_weeks,median_playtime_forever,median_playtime_two_weeks,developers,publishers,categories,genres,tags,screenshots,movies,n_screens,n_movies,n_tags,n_languages
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,{'English'},[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.galacticbowling.net,NaN,NaN,True,False,False,0,NaN,0,6,11,NaN,30,0,NaN,0,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,10,1,4,1
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"{ 'Japanese', 'Simplified Chinese', 'Traditi...",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://trainbandit.com,NaN,support@rustymoyher.com,True,True,False,0,NaN,0,53,5,NaN,12,0,NaN,0,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,5,1,20,10
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"{ 'Japanese', 'Simplified Chinese', 'Traditi...",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,https://henosisgame.com/,https://henosisgame.com/,info@henosisgame.com,True,True,True,0,NaN,0,3,0,NaN,0,0,NaN,0,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,7,1,19,11
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"{'English', 'Spanish - Spain'}",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,https://www.unusual-games.com/home/,https://www.unusual-games.com/contact/,welistentoyou@unusual-games.com,True,True,False,0,NaN,0,50,8,NaN,17,0,This Game may contain content not appropriate ...,0,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,24,1,6,2
5,1469160,Wartune Reborn,"Feb 26, 2021",50000 - 100000,68,0,0.00,0,Feel tired of auto-fight? Feel tired of boring...,{'English'},[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,https://7.wan.com,https://wartune@7road.com,True,False,False,0,NaN,0,87,49,NaN,0,0,NaN,0,0,0,0,7Road,7Road,"Single-player,Multi-player,MMO,PvP,Online PvP,...","Adventure,Casual,Free to Play,Massively Multip...","Turn-Based Combat,Massively Multiplayer,Multip...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,12,1,20,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85077,2704060,Ant Farm Simulator,"Jan 5, 2024",0 - 20000,3,0,0.99,0,Ant Farm (formicarium) With A Colony Of Ants. ...,"{ 'Japanese', 'Vietnamese', 'Ukrainian', 'P...",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,smirna.simulator.games@gmail.com,True,False,False,0,NaN,0,1,1,NaN,0,0,NaN,0,0,0,0,Smirna Simulator Games,Smirna Simulator Games,Single-player,"Casual,Indie,Simulatio

In [35]:
X_transformed = steam_df
X_transformed["supported_languages"] = X_transformed["supported_languages"].apply(
    lambda x: x.replace("[", "").replace("]", "")
)  # Change string format
X_transformed["supported_languages"] = X_transformed["supported_languages"].apply(
    lambda x: set(x.split(","))
)  # Transform into a set
X_transformed["n_languages"] = X_transformed["supported_languages"].apply(
    lambda n: len(n)
)

AttributeError: 'set' object has no attribute 'replace'

In [36]:
steam_df["screenshots"].apply(lambda x: len(set(x.split(","))))

0        10
1         5
2         6
3         7
4        24
         ..
85098     5
85099     5
85100     5
85101    15
85102    10
Name: screenshots, Length: 78590, dtype: int64

In [41]:
X_transformed = steam_df


X_transformed["n_screens"] = X_transformed["screenshots"].apply(
    lambda x: len(set(x.split(",")))
)
X_transformed["n_movies"] = X_transformed["movies"].apply(
    lambda x: len(set(x.split(",")))
)
X_transformed["n_tags"] = X_transformed["tags"].apply(lambda x: len(set(x.split(","))))

AttributeError: 'float' object has no attribute 'split'

In [12]:
X_transformed

,appid,name,release_date,estimated_owners,peak_ccu,required_age,price,dlc_count,about_the_game,supported_languages,full_audio_languages,reviews,header_image,website,support_url,support_email,windows,mac,linux,metacritic_score,metacritic_url,user_score,positive,negative,score_rank,achievements,recommendations,notes,average_playtime_forever,average_playtime_two_weeks,median_playtime_forever,median_playtime_two_weeks,developers,publishers,categories,genres,tags,screenshots,movies,n_languages
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,{'English'},[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.galacticbowling.net,NaN,NaN,True,False,False,0,NaN,0,6,11,NaN,30,0,NaN,0,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,1
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"{ 'Italian', 'French', 'German', 'Tradition...",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://trainbandit.com,NaN,support@rustymoyher.com,True,True,False,0,NaN,0,53,5,NaN,12,0,NaN,0,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,10
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"{ 'Portuguese - Brazil', 'English'}",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,ramoncampiaof31@gmail.com,True,False,False,0,NaN,0,0,0,NaN,0,0,NaN,0,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,2
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"{ 'Italian', 'French', 'German', 'Tradition...",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,https://henosisgame.com/,https://henosisgame.com/,info@henosisgame.com,True,True,True,0,NaN,0,3,0,NaN,0,0,NaN,0,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,11
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"{ 'Spanish - Spain', 'English'}",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,https://www.unusual-games.com/home/,https://www.unusual-games.com/contact/,welistentoyou@unusual-games.com,True,True,False,0,NaN,0,50,8,NaN,17,0,This Game may contain content not appropriate ...,0,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85098,2669080,Mannerheim's Saloon Car,"Jan 2, 2024",0 - 0,0,0,0.00,0,Marshal Mannerheim’s Saloon Car is the train c...,"{'English', 'Finnish'}",['Finnish'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,xgsmikkeli@gmail.com,True,False,False,0,NaN,0,0,0,NaN,0,0,NaN,0,0,0,0,Xamk Game Studios,"Sodan ja rauhan keskus Muisti, Päämajamuseo","Single-player,Tracked Controller Support,VR Only","Adventure,Simulation",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,2
85099,2736910,Beer Run,"Ja

In [27]:
numeric_columns = steam_df.select_dtypes(include=[float, int, bool]).columns

In [43]:
steam_df_2023 = steam_df[steam_df["year"] >= 2023]

In [44]:
steam_df_2023[numeric_columns].groupby(by="success_1000_threshold").median()

,Peak CCU,Required age,Price,DLC count,Windows,Mac,Linux,Metacritic score,User score,Positive,Negative,Score rank,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,about_length,n_screens,n_movies,n_tags,n_languages,has_publisher,perceived_quality,no_user_reactions,total_reviews,has_support_email,has_support_url,has_website,month,day,year,success_500_threshold
success_1000_threshold,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,4.79,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1045.0,7.0,1.0,1.0,1.0,0.0,0.000000,1.0,0.0,1.0,0.0,0.0,7.0,16.0,2023.0,0.0
1,2278.0,0.0,24.99,1.0,1.0,0.0,0.0,0.0,0.0,1859.0,360.0,NaN,38.0,2068.5,354.5,134.5,356.5,157.5,1716.0,10.0,2.0,20.0,11.0,1.0,5.646919,0.0,2421.0,1.0,1.0,1.0,9.0,16.0,2023.0,1.0


## Limiting the dataset

In [56]:
steam_df_indie = steam_df_2023[
    steam_df_2023["Genres"].str.contains("Indie").fillna(False)
].sort_values(by="total_reviews", ascending=False)
steam_df_indie = steam_df_indie[steam_df_indie["Metacritic score"] == 0]
steam_df_indie = steam_df_indie[steam_df_indie["total_reviews"] > 10]

In [57]:
steam_df_indie = steam_df_indie[steam_df_indie["total_reviews"] > 10]

In [59]:
steam_df_indie[numeric_columns].groupby(by="success_500_threshold").median()

,Peak CCU,Required age,Price,DLC count,Windows,Mac,Linux,Metacritic score,User score,Positive,Negative,Score rank,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,about_length,n_screens,n_movies,n_tags,n_languages,has_publisher,perceived_quality,no_user_reactions,total_reviews,has_support_email,has_support_url,has_website,month,day,year,success_1000_threshold
success_500_threshold,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2.0,0.0,8.99,0.0,1.0,0.0,0.0,0.0,0.0,25.0,3.0,NaN,12.0,0.0,0.0,0.0,0.0,0.0,1334.0,8.0,1.0,20.0,2.0,0.0,7.500000,0.0,29.0,1.0,0.0,0.0,8.0,16.0,2023.0,0.0
1,432.0,0.0,17.99,0.0,1.0,0.0,0.0,0.0,0.0,847.5,130.5,NaN,28.0,957.0,115.0,2.5,127.5,2.5,1695.5,10.0,2.0,20.0,7.0,1.0,8.594837,0.0,996.0,1.0,1.0,1.0,9.0,16.0,2023.0,0.0


##Having a publisher makes any difference?

In [91]:
steam_df_indie.groupby(by="Publishers").mean(numeric_only=True)

,Peak CCU,Required age,Price,DLC count,Windows,Mac,Linux,Metacritic score,User score,Positive,Negative,Score rank,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,about_length,n_screens,n_movies,n_tags,n_languages,has_publisher,perceived_quality,no_user_reactions,total_reviews,has_support_email,has_support_url,has_website,month,day,year,success_1000_threshold,success_500_threshold
Publishers,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
(Myco),1307.0,0.0,6.29,0.0,1.0,0.0,0.0,0.0,0.0,267.0,5.0,NaN,17.0,334.0,114.0,114.0,114.0,114.0,1499.0,10.0,1.0,20.0,1.0,1.0,44.500000,0.0,272.0,1.0,0.0,1.0,12.0,14.0,2023.0,0.0,0.0
-乌鸦男-,1.0,0.0,0.99,0.0,1.0,0.0,0.0,0.0,0.0,108.0,10.0,NaN,0.0,116.0,0.0,0.0,0.0,0.0,424.0,6.0,1.0,20.0,1.0,0.0,9.818182,0.0,118.0,1.0,1.0,0.0,8.0,8.0,2023.0,0.0,0.0
001,2.0,0.0,1.29,0.0,1.0,0.0,0.0,0.0,0.0,36.5,2.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1337.5,7.0,1.0,20.0,3.0,0.0,13.700000,0.0,38.5,1.0,1.0,1.0,4.5,6.5,2023.0,0.0,0.0
0UP GAMES,205.0,0.0,15.29,0.0,1.0,0.0,0.0,0.0,0.0,348.0,2.0,NaN,14.0,371.0,0.0,0.0,0.0,0.0,1133.0,20.0,1.0,20.0,2.0,1.0,116.000000,0.0,350.0,1.0,0.0,0.0,2.0,20.0,2023.0,0.0,0.0
"100 GAMES,Gamersky Games",0.0,0.0,10.49,0.0,1.0,0.0,0.0,0.0,0.0,54.0,15.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1195.0,10.0,1.0,12.0,15.0,1.0,3.375000,0.0,69.0,1.0,0.0,0.0,3.0,17.0,2023.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
野生建模师,0.0,0.0,0.99,0.0,1.0,0.0,0.0,0.0,0.0,31.0,36.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,326.0,5.0,1.0,20.0,8.0,0.0,0.837838,0.0,67.0,1.0,0.0,0.0,8.0,29.0,2023.0,0.0,0.0
"阿地,Miradil—DHU",2.0,0.0,6.99,0.0,1.0,0.0,0.0,0.0,0.0,33.0,21.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,2465.0,12.0,2.0,10.0,1.0,0.0,1.500000,0.0,54.0,1.0,1.0,0.0,4.0,11.0,2023.0,0.0,0.0
阿基米德 尼古拉斯 夏,0.0,0.0,2.39,0.0,1.0,0.0,0.0,0.0,0.0,21.0,8.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,750.0,9.0,1.0,19.0,2.0,1.0,2.333333,0.0,29.0,1.0,1.0,0.0,1.0,26.0,2023.0,0.0,0.0


In [87]:
list(numeric_columns).append("Publisher")
# s

In [88]:
numeric_columns

Index(['Peak CCU', 'Required age', 'Price', 'DLC count', 'Windows', 'Mac',
       'Linux', 'Metacritic score', 'User score', 'Positive', 'Negative',
       'Score rank', 'Achievements', 'Recommendations',
       'Average playtime forever', 'Average playtime two weeks',
       'Median playtime forever', 'Median playtime two weeks', 'about_length',
       'n_screens', 'n_movies', 'n_tags', 'n_languages', 'has_publisher',
       'perceived_quality', 'no_user_reactions', 'total_reviews',
       'has_support_email', 'has_support_url', 'has_website', 'month', 'day',
       'year', 'success_1000_threshold', 'success_500_threshold'],
      dtype='object')

In [95]:
sys.path.append("/Users/matheus/Projects/steam_analysis/steam-analysis/src/features")

In [5]:
from src.features.build_features import SteamDataTransformer

In [6]:
transformer = SteamDataTransformer()

In [7]:
transformer.fit(steam_df)

SteamDataTransformer()

In [8]:
teste = transformer.transform(steam_df)

In [9]:
teste

,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Full audio languages,...,no_user_reactions,total_reviews,has_support_email,has_support_url,has_website,month,day,year,success_1000_threshold,success_500_threshold
0,Galactic Bowling,2008-10-21,0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,"{{""'English'""}}",[],...,False,17,False,False,True,10,21,2008,0,0
1,Train Bandit,2017-10-12,0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"{ ""'English'"", "" 'Italian'""}, "" 'Portuguese ...",[],...,False,58,True,False,True,10,12,2017,0,0
2,Jolt Project,2021-11-17,0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"{ ""'English'""}, {"" 'Portuguese - Brazil'""}",[],...,True,0,True,False,False,11,17,2021,0,0
3,Henosis™,2020-07-23,0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"{ "" 'Portuguese'"", "" 'Korean'"", ""'English'"",...",[],...,False,3,True,True,True,7,23,2020,0,0
4,Two Weeks in Painland,2020-02-03,0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"{ ""'English'""}, {"" 'Spanish - Spain'""}",[],...,False,58,True,True,True,2,3,2020,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78585,Mannerheim's Saloon Car,2024-01-02,0 - 0,0,0,0.00,0,Marshal Mannerheim’s Saloon Car is the train c...,"{ ""'English'""}, {"" 'Finnish'""}",['Finnish'],...,True,0,True,False,False,1,2,2024,0,0
78586,Beer Run,2024-01-03,0 - 0,0,0,0.00,0,Beer Run is an Indie game created to steal bee...,"{{""'English'""}}",[],...,True,0,True,False,False,1,3,2024,0,0
78587,My Friend The Spider,2024-01-04,0 - 0,0,0,0.00,0,A small 'horror' narrative game about isolatio...,"{{""'English'""}}",['English'],...,True,0,False,True,False,1,4,2024,0,0
78588,Path of Survivors,2024-01-08,0 - 0,0,0,3.99,0,Path of Survivors is a multi-class auto-battle...,"{{""'English'""}}",[],...,True,0,True,True,True,1,8,2024,0,0


In [21]:
X_transformed = steam_df.copy()
X_transformed = X_transformed.dropna(subset=["Screenshots"])
X_transformed = X_transformed.dropna(subset=["Movies"])
X_transformed = X_transformed.dropna(subset=["Genres"])
X_transformed = X_transformed.dropna(subset=["Genres"])
X_transformed = X_transformed["Tags"].fillna("empty")
X_transformed = X_transformed["About the game"].fillna("empty")
X_transformed = X_transformed.drop(columns="AppID")

KeyError: 'About the game'

In [10]:
steam_df.dropna(subset=["Screenshots"])

,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Full audio languages,...,no_user_reactions,total_reviews,has_support_email,has_support_url,has_website,month,day,year,success_1000_threshold,success_500_threshold
0,Galactic Bowling,2008-10-21,0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,"{""'English'""}",[],...,False,17,False,False,True,10,21,2008,0,0
1,Train Bandit,2017-10-12,0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"{"" 'German'"", "" 'Russian'"", "" 'Japanese'"", "" '...",[],...,False,58,True,False,True,10,12,2017,0,0
2,Jolt Project,2021-11-17,0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"{"" 'Portuguese - Brazil'"", ""'English'""}",[],...,True,0,True,False,False,11,17,2021,0,0
3,Henosis™,2020-07-23,0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"{"" 'German'"", "" 'Korean'"", "" 'Portuguese'"", "" ...",[],...,False,3,True,True,True,7,23,2020,0,0
4,Two Weeks in Painland,2020-02-03,0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"{"" 'Spanish - Spain'"", ""'English'""}",[],...,False,58,True,True,True,2,3,2020,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78585,Mannerheim's Saloon Car,2024-01-02,0 - 0,0,0,0.00,0,Marshal Mannerheim’s Saloon Car is the train c...,"{"" 'Finnish'"", ""'English'""}",['Finnish'],...,True,0,True,False,False,1,2,2024,0,0
78586,Beer Run,2024-01-03,0 - 0,0,0,0.00,0,Beer Run is an Indie game created to steal bee...,"{""'English'""}",[],...,True,0,True,False,False,1,3,2024,0,0
78587,My Friend The Spider,2024-01-04,0 - 0,0,0,0.00,0,A small 'horror' narrative game about isolatio...,"{""'English'""}",['English'],...,True,0,False,True,False,1,4,2024,0,0
78588,Path of Survivors,2024-01-08,0 - 0,0,0,3.99,0,Path of Survivors is a multi-class auto-battle...,"{""'English'""}",[],...,True,0,True,True,True,1,8,2024,0,0
